In [1]:
import torch
import torch.nn as nn
import numpy as np
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

In [22]:
import sys
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2
from dataclasses import dataclass, field
from typing import List, Optional
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from glob import glob
import random
from tensorflow.keras.callbacks import Callback 
import json
from dataclasses import dataclass
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import BatchNormalization, Dropout, LeakyReLU
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
from concurrent.futures import ProcessPoolExecutor, as_completed
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [23]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results
# 定义需要显示的特定面部关键点编号
selected_indices = [
    419, 290, 303, 242, 56, 155, 221, 226, 387, 362, 385, 310, 295, 340, 0, 37, 39, 40, 178, 146, 90, 72, 
    448, 380, 274, 398, 87, 98, 64, 324, 222, 1, 13, 22, 159, 145, 157, 89, 
    312, 462, 259, 63, 66, 112, 461, 463, 348, 62, 308, 119, 269, 78, 16, 65, 144, 163, 
    384, 229, 84, 321, 325, 466, 403, 182, 232, 219, 141, 249, 196, 320, 95, 
    304, 77, 272, 224, 239, 268, 316, 405, 86, 186, 
    63, 296, 334, 53, 195, 66, 107, 52, 65
]

# 定义数据类来存储每个关键点的坐标
@dataclass
class Landmark:
    x: float
    y: float
    z: float

# 定义 CustomResults 数据类
@dataclass
class CustomResults:
    face_landmarks: Optional[landmark_pb2.NormalizedLandmarkList] = None
    pose_landmarks: Optional[landmark_pb2.NormalizedLandmarkList] = None
    left_hand_landmarks: Optional[landmark_pb2.NormalizedLandmarkList] = None
    right_hand_landmarks: Optional[landmark_pb2.NormalizedLandmarkList] = None

# 提取并过滤面部关键点并生成 NormalizedLandmarkList
def create_filtered_face_landmarks(landmarks, indices):
    if not landmarks:
        return None
    filtered_landmarks = [landmarks[idx] for idx in indices]
    return landmark_pb2.NormalizedLandmarkList(landmark=filtered_landmarks)
# 使用标准的 MediaPipe 绘图方式来绘制关键点（无连接线）
def draw_landmarks(image, custom_results):
    h, w, _ = image.shape
    
    # 绘制面部关键点（不包含连接线）
    if custom_results.face_landmarks:
        for idx, landmark in zip(selected_indices, custom_results.face_landmarks.landmark):
            x, y = int(landmark.x * w), int(landmark.y * h)
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)  
            cv2.putText(image, str(idx), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1, cv2.LINE_AA)

    # 绘制姿势关键点
    if custom_results.pose_landmarks:
        mp_drawing.draw_landmarks(
            image, custom_results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(88, 22, 88), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(66, 22, 66), thickness=2, circle_radius=2)
        )

    # 绘制左手关键点
    if custom_results.left_hand_landmarks:
        mp_drawing.draw_landmarks(
            image, custom_results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(100, 22, 200), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(100, 22, 200), thickness=2, circle_radius=2)
        )

    # 绘制右手关键点
    if custom_results.right_hand_landmarks:
        mp_drawing.draw_landmarks(
            image, custom_results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 90, 0), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(0, 90, 0), thickness=2, circle_radius=2)
        )

def calculate_velocity_acceleration(data):
    """
    计算每个帧的速度和加速度。
    
    参数：
    - data: 形状为 (num_samples, sequence_length, num_keypoints) 的 3D 数组

    返回：
    - 带有速度和加速度特征的新数据，形状为 (num_samples, sequence_length, num_keypoints * 3)
    """
    # 计算速度
    velocity = np.diff(data, axis=1)  # 对每个时间步进行差分，得到速度
    velocity = np.concatenate([velocity[:, :1, :], velocity], axis=1)  # 保持与原始帧数量一致
    
    # 计算加速度
    acceleration = np.diff(velocity, axis=1)  # 对速度再进行差分，得到加速度
    acceleration = np.concatenate([acceleration[:, :1, :], acceleration], axis=1)  # 保持与原始帧数量一致
    
    return velocity, acceleration
def calculate_relative_change(features):
    """
    计算每一帧相对于前一帧的相对变化量。
    
    参数：
    - features: 形状为 (num_samples, sequence_length, num_keypoints) 的 3D 数组

    返回：
    - 相对变化的特征
    """
    # 对每一帧计算相对变化
    relative_change = np.diff(features, axis=1)  # 计算相邻帧的差异
    relative_change = np.concatenate([relative_change[:, :1, :], relative_change], axis=1)  # 保持形状一致

    return relative_change

def extract_keypoints(custom_results):
    # 提取位置关键点 (face, pose, left_hand, right_hand)
    face = np.array([[lm.x, lm.y, lm.z] for lm in custom_results.face_landmarks.landmark]).flatten() \
        if custom_results.face_landmarks else np.zeros(90 * 3)
    pose = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in custom_results.pose_landmarks.landmark]).flatten() \
        if custom_results.pose_landmarks else np.zeros(33 * 4)
    lh = np.array([[lm.x, lm.y, lm.z] for lm in custom_results.left_hand_landmarks.landmark]).flatten() \
        if custom_results.left_hand_landmarks else np.zeros(21 * 3)
    rh = np.array([[lm.x, lm.y, lm.z] for lm in custom_results.right_hand_landmarks.landmark]).flatten() \
        if custom_results.right_hand_landmarks else np.zeros(21 * 3)

    # 位置数据合并
    position_features = np.concatenate([face, pose, lh, rh])

    # 中心化位置特征，以第 85 个关键点（例如身体中心点）为中心
    def centralize_keypoints(keypoints, center_idx=85):
        center_x = keypoints[center_idx * 3]
        center_y = keypoints[center_idx * 3 + 1]
        center_z = keypoints[center_idx * 3 + 2]
        
        keypoints[::3] -= center_x  # X 坐标中心化
        keypoints[1::3] -= center_y  # Y 坐标中心化
        keypoints[2::3] -= center_z  # Z 坐标中心化
        return keypoints

    position_features_centered = centralize_keypoints(position_features)

    # 计算速度和加速度（不进行归一化）
    velocity, acceleration = calculate_velocity_acceleration(position_features_centered.reshape(1, -1, 3))

    # 计算相对变化
    relative_velocity = calculate_relative_change(velocity)
    relative_acceleration = calculate_relative_change(acceleration)

    # 拼接位置、速度和加速度特征
    all_features = np.concatenate([
        position_features_centered,
        relative_velocity.flatten(),
        relative_acceleration.flatten()
    ])

    return all_features

In [2]:
import torch
import torch.nn as nn
import numpy as np
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from sklearn.model_selection import train_test_split


def ensure_sequence_shape(sequence):
    """
    确保输入的 sequence 形状为 (seq_len, n_keypoints, 3)。
    如果输入是 2D，将其重塑为 3D。
    """
    if sequence.dim() == 2 and sequence.shape[1] % 3 == 0:
        n_keypoints = sequence.shape[1] // 3
        sequence = sequence.view(sequence.shape[0], n_keypoints, 3)
    elif sequence.dim() != 3:
        raise ValueError(f"Expected sequence to have shape (seq_len, n_keypoints, 3), but got {sequence.shape}")
    return sequence
def compute_mean_std(X_train):
    """
    计算数据集 X_train 的全局均值和标准差。
    
    Args:
        X_train (list or np.ndarray): 包含所有训练样本的列表或数组。
    
    Returns:
        tuple: global_mean, global_std
    """
    all_keypoints = np.concatenate([sample for sample in X_train], axis=0)  # 将所有样本拼接在一起
    mean = np.nanmean(all_keypoints, axis=0)  # 忽略 NaN 计算均值
    std = np.nanstd(all_keypoints, axis=0)    # 忽略 NaN 计算标准差
    return mean, std
def temporal_resample(sequence, target_length=105, min_scale=0.5, max_scale=1.5):
    sequence = ensure_sequence_shape(sequence)  # 确保序列形状为 (seq_len, n_keypoints, 3)
    scale = np.random.uniform(min_scale, max_scale)
    new_length = int(sequence.shape[0] * scale)

    # 调整为 3D 输入
    n_keypoints = sequence.shape[1]
    sequence = sequence.permute(1, 0, 2).contiguous().view(-1, sequence.shape[0])  # (n_keypoints * 3, seq_len)
    
    # 重采样到 new_length
    resampled_sequence = F.interpolate(sequence.unsqueeze(0), size=new_length, mode='linear', align_corners=False).squeeze(0)
    
    # 确保重塑大小匹配，并恢复 (new_length, n_keypoints, 3) 的形状
    resampled_sequence = resampled_sequence.view(n_keypoints, new_length, 3).permute(1, 0, 2)

    # 最终调整到 target_length
    final_sequence = F.interpolate(resampled_sequence.permute(1, 2, 0).contiguous().view(-1, resampled_sequence.shape[0]).unsqueeze(0),
                                   size=target_length, mode='linear', align_corners=False).squeeze(0)
    
    # 恢复形状 (target_length, n_keypoints, 3)
    final_sequence = final_sequence.view(n_keypoints, target_length, 3).permute(1, 0, 2)
    return final_sequence

def windowed_cutmix(sequence1, sequence2, target_length=105):
    sequence1, sequence2 = ensure_sequence_shape(sequence1), ensure_sequence_shape(sequence2)
    cut_ratio = np.random.rand()
    cut_point1 = int(sequence1.shape[0] * cut_ratio)
    cut_point2 = int(sequence2.shape[0] * cut_ratio)
    mixed_sequence = torch.cat((sequence1[:cut_point1], sequence2[cut_point2:]), dim=0)

    # 调整为 3D 输入，应用 interpolate
    mixed_sequence = mixed_sequence.permute(1, 0, 2).contiguous()  # (n_keypoints, seq_len, 3)
    mixed_sequence = mixed_sequence.view(-1, mixed_sequence.shape[1])  # (n_keypoints * 3, seq_len)
    final_sequence = F.interpolate(mixed_sequence.unsqueeze(0), size=target_length, mode='linear', align_corners=False).squeeze(0)
    final_sequence = final_sequence.view(sequence1.shape[1], target_length, 3).permute(1, 0, 2)  # (target_length, n_keypoints, 3)
    return final_sequence
def temporal_shift(sequence, target_length=105, max_shift=10):
    sequence = ensure_sequence_shape(sequence)  # 确保形状为 (seq_len, n_keypoints, 3)
    shift = np.random.randint(-max_shift, max_shift)
    shifted_sequence = torch.roll(sequence, shifts=shift, dims=0)

    # 最终调整为 target_length
    seq_len = shifted_sequence.shape[0]
    if seq_len > target_length:
        shifted_sequence = F.interpolate(shifted_sequence.permute(1, 2, 0).unsqueeze(0), size=target_length, mode='linear', align_corners=False)
        shifted_sequence = shifted_sequence.squeeze(0).permute(2, 0, 1)
    elif seq_len < target_length:
        padding = torch.zeros((target_length - seq_len, shifted_sequence.shape[1], shifted_sequence.shape[2]), dtype=shifted_sequence.dtype)
        shifted_sequence = torch.cat((shifted_sequence, padding), dim=0)
    
    return shifted_sequence

def random_keypoint_dropout(sequence, num_points_to_drop=6, num_time_windows=3):
    sequence = ensure_sequence_shape(sequence)  # 确保形状为 (seq_len, n_keypoints, 3)
    seq_len, num_keypoints, _ = sequence.shape
    for _ in range(num_time_windows):
        start = np.random.randint(0, seq_len)
        end = min(seq_len, start + np.random.randint(1, seq_len // num_time_windows))
        drop_indices = np.random.choice(num_keypoints, num_points_to_drop, replace=False)
        sequence[start:end, drop_indices, :] = 0  # 将选定关键点置为 0
    return sequence

def spatial_mask(sequence, mask_prob=0.3, max_points=10):
    sequence = ensure_sequence_shape(sequence)  # 确保形状为 (seq_len, n_keypoints, 3)
    if np.random.rand() < mask_prob:
        num_keypoints = sequence.shape[1]
        mask_points = np.random.choice(num_keypoints, max_points, replace=False)
        sequence[:, mask_points, :] = 0  # 空间遮挡，将选定的关键点置为 0
    return sequence

def temporal_mask(sequence, mask_prob=0.3, max_mask_len=10):
    sequence = ensure_sequence_shape(sequence)  # 确保形状为 (seq_len, n_keypoints, 3)
    if np.random.rand() < mask_prob:
        seq_len = sequence.shape[0]
        mask_len = np.random.randint(1, max_mask_len)
        start = np.random.randint(0, seq_len - mask_len)
        sequence[start:start + mask_len, :, :] = 0  # 时间遮挡，将选定时间窗口内的关键点置为 0
    return sequence

def drop_face_or_pose(sequence, drop_face_prob=0.2, drop_pose_prob=0.2, face_indices=None, pose_indices=None):
    sequence = ensure_sequence_shape(sequence)  # 确保形状为 (seq_len, n_keypoints, 3)
    if np.random.rand() < drop_face_prob and face_indices is not None:
        sequence[:, face_indices, :] = 0  # 面部关键点置为 0
    if np.random.rand() < drop_pose_prob and pose_indices is not None:
        sequence[:, pose_indices, :] = 0  # 姿态关键点置为 0
    return sequence

def drop_hand_keypoints(sequence, drop_hand_prob=0.05, left_hand_indices=None, right_hand_indices=None):
    sequence = ensure_sequence_shape(sequence)  # 确保形状为 (seq_len, n_keypoints, 3)
    if np.random.rand() < drop_hand_prob:
        if left_hand_indices is not None:
            sequence[:, left_hand_indices, :] = 0  # 左手关键点置为 0
        if right_hand_indices is not None:
            sequence[:, right_hand_indices, :] = 0  # 右手关键点置为 0
    return sequence

def flip_keypoints(sequence, left_hand_indices, right_hand_indices):
    """
    仅对左右手的关键点进行翻转。

    Args:
        sequence (torch.Tensor): 形状为 (seq_len, n_keypoints, 3) 的关键点序列。
        left_hand_indices (list): 左手关键点索引。
        right_hand_indices (list): 右手关键点索引。
    
    Returns:
        torch.Tensor: 翻转后的关键点序列。
    """
    sequence = ensure_sequence_shape(sequence)  # 确保形状为 (seq_len, n_keypoints, 3)

    # 翻转 X 坐标
    sequence[:, :, 0] = -sequence[:, :, 0]

    # 创建副本，用于交换左右手关键点
    flipped_sequence = sequence.clone()
    for l_idx, r_idx in zip(left_hand_indices, right_hand_indices):
        flipped_sequence[:, r_idx, :] = sequence[:, l_idx, :]  # 左手 → 右手
        flipped_sequence[:, l_idx, :] = sequence[:, r_idx, :]  # 右手 → 左手

    return flipped_sequence
def compute_velocity_and_acceleration(sequence):
    """
    计算给定序列的速度和加速度。
    Args:
        sequence (torch.Tensor): 形状为 (seq_len, n_keypoints, 3) 的位置序列。
    Returns:
        torch.Tensor, torch.Tensor: 分别返回速度和加速度，形状均为 (seq_len, n_keypoints, 3)。
    """
    sequence = ensure_sequence_shape(sequence)  # 确保输入是 (seq_len, n_keypoints, 3)
    
    # 计算速度 (Δx, Δy, Δz)
    velocity = torch.diff(sequence, dim=0, prepend=sequence[:1])
    
    # 计算加速度 (Δv_x, Δv_y, Δv_z)
    acceleration = torch.diff(velocity, dim=0, prepend=velocity[:1])
    
    return velocity, acceleration


def affine_transform(sequence, scale_range=(0.9, 1.1), translation_range=(-0.1, 0.1), rotation_range=(-10, 10)):
    sequence = ensure_sequence_shape(sequence)  # 确保形状为 (seq_len, n_keypoints, 3)

    # 随机缩放
    scale = np.random.uniform(*scale_range)
    sequence = sequence * scale

    # 随机平移
    translation = np.random.uniform(*translation_range, size=(1, sequence.shape[1], sequence.shape[2]))
    sequence = sequence + torch.tensor(translation, dtype=sequence.dtype, device=sequence.device)

    # 随机旋转（围绕 Z 轴旋转）
    angle = np.radians(np.random.uniform(*rotation_range))
    rotation_matrix = torch.tensor([
        [np.cos(angle), -np.sin(angle), 0],
        [np.sin(angle),  np.cos(angle), 0],
        [0, 0, 1]
    ], dtype=sequence.dtype, device=sequence.device)
    sequence = torch.matmul(sequence, rotation_matrix)
    return sequence

# 在3D形状的 (seq_len, n_keypoints, 3) 中，定义关键点索引范围
face_indices = list(range(0, 90))            # 面部关键点在 3D 中为 0~89
pose_indices = list(range(90, 90 + 33))      # 姿态关键点在 3D 中为 90~122
left_hand_indices = list(range(123, 123 + 21))  # 左手关键点在 3D 中为 123~143
right_hand_indices = list(range(144, 144 + 21)) # 右手关键点在 3D 中为 144~164

DATA_PATH = "I:\\Ece496\\custom_data\\preprocess"
target_length = 105  # 设置目标长度

# 加载数据
X = np.load(os.path.join(DATA_PATH, "X.npy"), allow_pickle=True)
y = np.load(os.path.join(DATA_PATH, "y.npy"), allow_pickle=True)

# 划分数据集
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 保存划分后的数据
np.save(os.path.join(DATA_PATH, "X_train.npy"), X_train)
np.save(os.path.join(DATA_PATH, "y_train.npy"), y_train)
np.save(os.path.join(DATA_PATH, "X_val.npy"), X_val)
np.save(os.path.join(DATA_PATH, "y_val.npy"), y_val)
np.save(os.path.join(DATA_PATH, "X_test.npy"), X_test)
np.save(os.path.join(DATA_PATH, "y_test.npy"), y_test)

# 定义自定义数据集类
class ASLDataset(Dataset):
    def __init__(self, X, y, global_mean, global_std, augment=True):
        self.X = X
        self.y = y
        self.global_mean = global_mean
        self.global_std = global_std
        self.augment = augment

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        sequence = self.X[idx]
        label = self.y[idx]

        sequence = np.nan_to_num(sequence, nan=0.0)
        
        # 归一化和中心化
        sequence = (sequence - self.global_mean) / self.global_std
        sequence = torch.from_numpy(sequence).float()

        # 确保形状为 (seq_len, n_keypoints, 3)
        sequence = ensure_sequence_shape(sequence)

        # 应用数据增强
        if self.augment:
            sequence = apply_random_augmentations(sequence, target_length=target_length)

        return sequence, label

def apply_random_augmentations(sequence, augment_prob=0.5, resize_prob=0.8, target_length=105):
    """
    对序列数据应用增强操作，支持未展平的 3D 数据。
    sequence 形状: (seq_len, n_keypoints, feature_dim)
    """
    sequence = ensure_sequence_shape(sequence)  # 确保序列为 3D 形状 (seq_len, n_keypoints, 3)
    print(f"Initial sequence shape: {sequence.shape}")

    # 1. 初始计算速度和加速度（不添加到序列中）
    initial_velocity, initial_acceleration = compute_velocity_and_acceleration(sequence)

    # 2. 时间增强
    if np.random.rand() < augment_prob:
        sequence = temporal_resample(sequence, target_length=target_length)
        print(f"Sequence shape after temporal_resample: {sequence.shape}")
        # 重新计算速度和加速度
        velocity, acceleration = compute_velocity_and_acceleration(sequence)

    if np.random.rand() < augment_prob:
        sequence = temporal_shift(sequence)
        print(f"Sequence shape after temporal_shift: {sequence.shape}")
        # 重新计算速度和加速度
        velocity, acceleration = compute_velocity_and_acceleration(sequence)

    # 3. 空间增强
    if np.random.rand() < augment_prob:
        sequence = affine_transform(sequence)
        print(f"Sequence shape after affine_transform: {sequence.shape}")
        # 重新计算速度和加速度
        velocity, acceleration = compute_velocity_and_acceleration(sequence)

    # 4. 遮罩增强（无需重新计算速度和加速度）
    if np.random.rand() < augment_prob:
        sequence = drop_face_or_pose(sequence, face_indices=face_indices, pose_indices=pose_indices)
        print(f"Sequence shape after drop_face_or_pose: {sequence.shape}")
    if np.random.rand() < augment_prob:
        sequence = drop_hand_keypoints(sequence, left_hand_indices=left_hand_indices, right_hand_indices=right_hand_indices)
        print(f"Sequence shape after drop_hand_keypoints: {sequence.shape}")
    if np.random.rand() < augment_prob:
        sequence = spatial_mask(sequence)
        print(f"Sequence shape after spatial_mask: {sequence.shape}")
    if np.random.rand() < augment_prob:
        sequence = temporal_mask(sequence)
        print(f"Sequence shape after temporal_mask: {sequence.shape}")

    # 5. 翻转（需要重新计算速度和加速度）
    if np.random.rand() < augment_prob:
        sequence = flip_keypoints(sequence, left_hand_indices, right_hand_indices)
        print(f"Sequence shape after flip_keypoints: {sequence.shape}")
        # 重新计算速度和加速度
        velocity, acceleration = compute_velocity_and_acceleration(sequence)

    # 6. 最后将完整的速度和加速度加入序列
    if 'velocity' in locals() and 'acceleration' in locals():
        sequence = torch.cat((sequence, velocity, acceleration), dim=2)
    else:
        # 如果没有动态重新计算，使用初始速度和加速度
        sequence = torch.cat((sequence, initial_velocity, initial_acceleration), dim=2)
    
    print(f"Final sequence shape: {sequence.shape}")
    
    return sequence





# 计算均值和标准差用于归一化
global_mean, global_std = compute_mean_std(X_train)



# 创建数据集和数据加载器
train_dataset = ASLDataset(X_train, y_train, global_mean, global_std, augment=True)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# # 训练循环中应用增强并打印处理信息
# for epoch in range(1):  # 设定单个 epoch 测试
#     print(f"Epoch {epoch + 1}")
#     for batch_idx, (sequences, labels) in enumerate(train_loader):
#         print(f"Batch {batch_idx + 1}:")
#         for sample_idx, sequence in enumerate(sequences):
#             print(f"  Sample {sample_idx + 1} after augmentation has shape: {sequence.shape}")
#         break  # 只打印一个批次，确保代码运行后不输出过多信息
#     break


In [20]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, LeakyReLU, Masking
from tensorflow.keras.utils import Sequence
import numpy as np
from sklearn.model_selection import train_test_split
import os

# 数据路径
DATA_PATH = "I:\\Ece496\\custom_data\\preprocess"

# 加载原始数据
X = np.load(f"{DATA_PATH}/X.npy", allow_pickle=True)
y = np.load(f"{DATA_PATH}/y.npy", allow_pickle=True)

# 替换 NaN 为 0
X = np.nan_to_num(X, nan=0.0)

# 数据集划分
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 保存划分后的数据
np.save(f"{DATA_PATH}/X_train.npy", X_train)
np.save(f"{DATA_PATH}/y_train.npy", y_train)
np.save(f"{DATA_PATH}/X_val.npy", X_val)
np.save(f"{DATA_PATH}/y_val.npy", y_val)
np.save(f"{DATA_PATH}/X_test.npy", X_test)
np.save(f"{DATA_PATH}/y_test.npy", y_test)

# 自定义数据生成器
class ASLDataGenerator(Sequence):
    def __init__(self, X, y, batch_size, augment_fn=None):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.augment_fn = augment_fn  # 数据增强函数
    
    def __len__(self):
        return int(np.ceil(len(self.X) / self.batch_size))
    
    def __getitem__(self, idx):
        batch_X = self.X[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

        # 应用增强（如果提供了增强函数）
        if self.augment_fn:
            batch_X = np.array([self.augment_fn(x) for x in batch_X])
        
        return batch_X, batch_y

# 模型定义
def build_model(input_shape, num_classes):
    model = Sequential([
        Masking(mask_value=0.0, input_shape=input_shape),  # 添加 Masking 层，忽略填充值 0.0
        LSTM(128, return_sequences=True, activation='tanh'),
        Dropout(0.2),
        LSTM(64, return_sequences=True, activation='tanh'),
        Dropout(0.2),
        LSTM(32, activation='tanh'),
        Dense(64),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dense(32),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# 加载已保存的数据
X_train = np.load(f"{DATA_PATH}/X_train.npy")
y_train = np.load(f"{DATA_PATH}/y_train.npy")
X_val = np.load(f"{DATA_PATH}/X_val.npy")
y_val = np.load(f"{DATA_PATH}/y_val.npy")

# 数据检查
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")

# 数据生成器实例化
batch_size = 8
train_generator = ASLDataGenerator(X_train, y_train, batch_size=batch_size, augment_fn=None)
val_generator = ASLDataGenerator(X_val, y_val, batch_size=batch_size, augment_fn=None)

# 检查生成器输出形状
for sequences, labels in train_generator:
    print(f"Sequences shape: {sequences.shape}, Labels shape: {labels.shape}")
    break

# 模型实例化
input_shape = (X_train.shape[1], X_train.shape[2])  # 假设数据是 (样本数, 时间步长, 特征数)
num_classes = len(np.unique(y_train))
model = build_model(input_shape, num_classes)

# 检查模型结构
model.summary()

# 模型训练
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=500,  # 自定义 epoch 数量
    verbose=1
)


X_train shape: (408, 105, 528), y_train shape: (408,)
X_val shape: (51, 105, 528), y_val shape: (51,)
Sequences shape: (8, 105, 528), Labels shape: (8,)
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, 105, 528)          0         
                                                                 
 lstm_12 (LSTM)              (None, 105, 128)          336384    
                                                                 
 dropout_9 (Dropout)         (None, 105, 128)          0         
                                                                 
 lstm_13 (LSTM)              (None, 105, 64)           49408     
                                                                 
 dropout_10 (Dropout)        (None, 105, 64)           0         
                                                                 
 lstm_14 (LSTM)              (Non

In [21]:
model.save_weights(r"I:\Ece496\custom_data\preprocess\weight1_with_velocity.h5")

In [24]:
TEST_VIDEO_FOLDER = "I:\\Ece496\\custom_data\\realtime_test"
video_files = glob(os.path.join(TEST_VIDEO_FOLDER, "*.mp4"))

model.load_weights(r"I:\Ece496\custom_data\preprocess\weight1_with_velocity.h5")
mp_holistic = mp.solutions.holistic

# 使用 MediaPipe 处理视频
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for video_file in video_files:
        video_name = os.path.basename(video_file)
        print(f"Processing video: {video_name}")

        cap = cv2.VideoCapture(video_file)
        sequence = []  # 存储每帧的关键点
        predicted_labels = []  # 存储每帧的预测标签

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # 转换颜色空间并处理帧
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = holistic.process(frame_rgb)  # 使用已转换的RGB帧

            # 创建 custom_results 字典，确保传入正确的参数
            # 创建 custom_results 字典，确保传入正确的参数
            # Create custom results with filtered landmarks
            custom_results = CustomResults(
                face_landmarks=create_filtered_face_landmarks(results.face_landmarks.landmark, selected_indices) if results.face_landmarks else None,
                pose_landmarks=results.pose_landmarks,
                left_hand_landmarks=results.left_hand_landmarks,
                right_hand_landmarks=results.right_hand_landmarks
            )

            # Extract keypoints and ensure correct input shape
            keypoints = extract_keypoints(custom_results)


            if keypoints is not None:
                sequence.append(keypoints)

            # 检查是否达到 105 帧
            if len(sequence) == 105:
                prediction = model.predict(np.expand_dims(sequence, axis=0), verbose=0)[0]
                label = np.argmax(prediction)
                predicted_labels.append(label)
                sequence = []  # 清空 sequence 以便下一段 105 帧

        cap.release()

        # 统计出现最多的标签作为最终预测结果
        if predicted_labels:
            final_prediction = max(set(predicted_labels), key=predicted_labels.count)
            print(f"Video '{video_name}' processed. Final predicted label: {final_prediction} (Action: {words[final_prediction]})\n")
        else:
            print(f"Video '{video_name}' processed. No prediction available.\n")


Processing video: I or me_1_alice.mp4


ValueError: in user code:

    File "c:\ProgramData\anaconda3\envs\tf2x_clone\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "c:\ProgramData\anaconda3\envs\tf2x_clone\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\ProgramData\anaconda3\envs\tf2x_clone\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "c:\ProgramData\anaconda3\envs\tf2x_clone\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "c:\ProgramData\anaconda3\envs\tf2x_clone\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\ProgramData\anaconda3\envs\tf2x_clone\lib\site-packages\keras\engine\input_